<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/roof_type_classifier.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/roof_type_classifier.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**

In [1]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/NHERI-SimCenter/BrailsPlusPlus to /tmp/pip-req-build-6d052ia6
  Running command git clone --filter=blob:none --quiet https://github.com/NHERI-SimCenter/BrailsPlusPlus /tmp/pip-req-build-6d052ia6
  Resolved https://github.com/NHERI-SimCenter/BrailsPlusPlus to commit 6aac5565871074aff8e1ee3fc69f3f0e2a9408db
  Preparing metadata (setup.py) ... done


## **Import ImageSet and Importer Methods of BRAILS++**

In [2]:
from brails.types.image_set import ImageSet
from brails import Importer

## **Load Aerial Imagery**
The `RoofShapeClassifier` class is designed to identify roof types of buildings based on aerial imagery. In this example, we will load images from the `examples/image_processor/images/satellite_easy` directory of BRAILS++ to showcase how to use the `RoofShapeClassifier`.

To load these images, you first need to initialize an `ImageSet` object, which will be assigned to aerial_images. This object can then load the target images by scanning the specified directory using the set_directory method. The second argument in set_directory is necessary for loading the images; otherwise, the default behavior of `ImageSet` is to simply set the directory to the string value provided in the first argument (in this case, `images/satellite`).

In [3]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName", "data": "Pinellas County, FL"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

usa_fp_class = importer.get_class("USA_FootprintScraper")
usa_fp_data = {"length": "ft"}
footprint_scraper = usa_fp_class(usa_fp_data)
usa_fp_inventory = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 20 buildings using a random seed value of 40:
small_inventory = usa_fp_inventory.get_random_sample(20, 40)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_aerial_class = importer.get_class("GoogleSatellite")
google_aerial = google_aerial_class()
images_aerial = google_aerial.get_images(
    small_inventory, "tmp/satellite/")


Searching for Pinellas County, FL...
Found Pinellas County, Florida, United States

Meshing the defined area...


Obtaining the number of buildings in each cell: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1176/1176 [00:12<00:00, 96.43it/s]



Meshing complete. Split Pinellas County into 1230 cells


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1230/1230 [00:00<00:00, 56157.82it/s]



Found a total of 345057 building footprints in Pinellas County


INFO:brails.scrapers.google_satellite.google_satellite:Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/satellite
Obtaining satellite imagery: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 43.59it/s]


## **Predict Roof Types of Buildings Using RoofShapeClassifier Module**
`RoofShapeClassifier` includes a pre-trained EfficientNet_V2_M model that has been trained on a custom dataset of 100,000 images. This model is capable of classifying roofs seen in aerial imagery into three categories: `Flat`, `Hip`, and `Gable`.

In [5]:
importer = Importer()
roof_shape_classifier_class = importer.get_class('RoofShapeClassifier')
roof_shape_classifier = roof_shape_classifier_class()
predictions = roof_shape_classifier.predict(images_aerial)


Default roof classifier model in tmp/models/roofTypeClassifier_v1.pth loaded


/home/bacetiner/anaconda3/envs/brails_plus/lib/python3.11/site-packages/brails/processors/image_classifier/image_classifier.py:734: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

## **Show Prediction Results**

In [6]:
print(predictions)

{240441: 'Gable', 303818: 'Gable', 274712: 'Gable', 16696: 'Hip', 128602: 'Gable', 148072: 'Hip', 334022: 'Flat', 108178: 'Gable', 67304: 'Flat', 182656: 'Gable', 145290: 'Flat', 231086: 'Gable', 15011: 'Gable', 276559: 'Flat', 68091: 'Hip', 31094: 'Hip', 336208: 'Gable', 105897: 'Hip', 324838: 'Gable', 241521: 'Gable'}
